In [1]:
#Import all the necesary libraries and Classifiers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashwi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashwi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ashwi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data = pd.read_csv('gender-classifier-DFE-791531.csv',encoding='latin1')

In [3]:
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [4]:
data.shape

(20050, 26)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

In [6]:
#Data Cleaning
#we will be droping the redundant columns from our data
data.drop(['_unit_id','_last_judgment_at','created','fav_number','profileimage','retweet_count','tweet_coord',
         '_trusted_judgments', 'tweet_count', 'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone', 
         '_golden','_unit_state', 'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'sidebar_color', 
         'profile_yn', 'profile_yn:confidence','gender:confidence'], axis=1, inplace=True)

In [7]:
data.head()

,gender,description,text
0,male,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,male,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...


In [8]:
#we will check for null values
data.isna().sum()

gender           97
description    3744
text              0
dtype: int64

In [9]:
#Will drop the rows with null values
data.dropna(axis = 0, inplace = True)

In [10]:
# count the gender column
data['gender'].value_counts()

gender
female     5725
male       5469
brand      4328
unknown     702
Name: count, dtype: int64

In [11]:
#we will only keep the male and female variables and drop the others.
data = data[(data.gender == 'female') | (data.gender == 'male')]

In [12]:
#And check the counts again
data['gender'].value_counts()

gender
female    5725
male      5469
Name: count, dtype: int64

In [13]:
#Data Preprocessing
#encode male and female category as 1 and 0
for gen in data['gender']:
    if gen == 'male':
        data['gender'].replace({'male':1}, inplace = True)
    elif gen == 'female':
        data['gender'].replace({'female':0}, inplace = True)

data['gender'].value_counts()

gender
0    5725
1    5469
Name: count, dtype: int64

In [14]:
#Cleaning Text
#we will be cleaning the description column
def clean(review):

    descrip = re.sub('[^a-zA-Z]', ' ', review)

    review = review.lower()
    return review

data['descrip_cleaned'] = pd.DataFrame(data['description'].apply(lambda x: clean(x)))

data.head()

,gender,description,text,descrip_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...


In [15]:
data['descrip_cleaned'].replace('[@+]', "", regex=True,inplace=True)
data['descrip_cleaned'].replace('[()]', "", regex=True,inplace=True)
data['descrip_cleaned'].replace('[#+]', "", regex=True,inplace=True)
url_regex = '''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
data['descrip_cleaned'] = data['descrip_cleaned'].replace(url_regex, "", regex=True)

In [16]:
data.head()

,gender,description,text,descrip_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...


In [17]:
#Tokenize the cleaned descriptions
data['descrip_cleaned'] = [nltk.word_tokenize(tweet) for tweet in data['descrip_cleaned']]
descrip_new = []
for each_row in data['descrip_cleaned']:
    descrip_new.append([i for i in each_row if i.isalpha()])
descrip_new_alpha = []

In [18]:
#Stopwords Removal
#stop words don't add any meaning to the sentences. we will remove them
stop_words = set(stopwords.words('english'))
# now remove them from the list
for each_row in descrip_new:
    descrip_new_alpha.append([i for i in each_row if i not in stop_words])

In [19]:
#Lemmatization
#Lemmatization is the process of grouping together the different inflected forms of a word so they can be analyzed as a single 
#item. Lemmatization is similar to stemming but it brings context to the words. 
#So it links words with similar meanings to one word.
description_new_lemma = []
lemma = nltk.WordNetLemmatizer()
for each_row in descrip_new_alpha:
    description_new_lemma.append([lemma.lemmatize(word) for word in each_row])
#put back the new sentences
data['descrip_cleaned'] = description_new_lemma
data['descrip_cleaned'] = [" ".join(desc) for desc in data['descrip_cleaned'].values]

In [20]:
#Text cleaning of the 'text' column
#whatever we've done for the 'description' column, will do that for 'text' column too
data['text_cleaned'] = pd.DataFrame(data['text'].apply(lambda y: clean(y)))
data.head()

,gender,description,text,descrip_cleaned,text_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,sing rhythm,robbie e responds to critics after win against...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,author novel filled family drama romance,ûïit felt like they were my friends and i was...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining squealing,i absolutely adore when louis starts the songs...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,mobile guy shazam google kleiner perkins yahoo...,hi @jordanspieth - looking at the url - do you...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson best chief best band xxxx thank k...,watching neighbours on sky+ catching up with t...


In [21]:
data['text_cleaned'].replace('[@+]', "", regex=True,inplace=True)
data['text_cleaned'].replace('[()]', "", regex=True,inplace=True)
data['text_cleaned'].replace('[#+]', "", regex=True,inplace=True)
url_regex = '''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
data['text_cleaned'] = data['text_cleaned'].replace(url_regex, "", regex=True)

In [22]:
data['text_cleaned'] = [nltk.word_tokenize(tweet) for tweet in data['text_cleaned']]
text_new = []
for each_row in data['text_cleaned']:
    text_new.append([i for i in each_row if i.isalpha()])
text_new_alpha = []

In [23]:
for each_row in text_new:
    text_new_alpha.append([i for i in each_row if i not in stop_words])

In [24]:
text_new_lemma = []
for each_row in text_new_alpha:
    text_new_lemma.append([lemma.lemmatize(word) for word in each_row])
#put back the new sentences
data['text_cleaned'] = text_new_lemma
data['text_cleaned'] = [" ".join(desc) for desc in data['text_cleaned'].values]

In [25]:
data.head()

,gender,description,text,descrip_cleaned,text_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,sing rhythm,robbie e responds critic win eddie edward worl...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,author novel filled family drama romance,felt like friend living story http retired http
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining squealing,absolutely adore louis start song hit hard fee...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,mobile guy shazam google kleiner perkins yahoo...,hi jordanspieth looking url use ifttt typicall...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson best chief best band xxxx thank k...,watching neighbour sky catching neighbs xxx xxx


In [26]:
#droping Redundant Columns description and text
data.drop(['description', 'text'], axis = 1, inplace = True)
data.head()

,gender,descrip_cleaned,text_cleaned
0,1,sing rhythm,robbie e responds critic win eddie edward worl...
1,1,author novel filled family drama romance,felt like friend living story http retired http
2,1,louis whining squealing,absolutely adore louis start song hit hard fee...
3,1,mobile guy shazam google kleiner perkins yahoo...,hi jordanspieth looking url use ifttt typicall...
4,0,ricky wilson best chief best band xxxx thank k...,watching neighbour sky catching neighbs xxx xxx


In [27]:
#vectorization of texts
#Text Vectorization is the process of converting text into numerical representation.
cv = CountVectorizer(max_features = 1000)
x = cv.fit_transform(data['descrip_cleaned']).toarray()
x1 = cv.fit_transform(data['text_cleaned']).toarray()


In [28]:
#let's save these two set of vectors in two different pandas dataframe
A = pd.DataFrame(x)
B = pd.DataFrame(x1)

In [29]:
#and then concatenate these two dataframes
X = pd.concat([B, A], join = 'outer', axis = 1)
X.shape

(11194, 2000)

In [30]:
#the shape of this new dataframe matches shape of our gender column
data['gender'].shape

(11194,)

In [31]:
#Now build our independent variables and Target variable
x = np.array(X)
y = np.array(data['gender'])

In [32]:
#Split the data into train test set
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 0)

In [33]:
#Classification
#we will use thrre classifier for this task. Naive Bayes, Lightgbm and XGBoost
#Naive Bayes
gnb_model = GaussianNB()
gnb_model.fit(X_train , y_train)
y_pred = gnb_model.predict(X_test)

In [34]:
#Evaluation
accuracy = accuracy_score(y_test, y_pred)
print("accuracy: %.2f%%" % (accuracy*100.0))

accuracy: 63.33%


In [35]:
confusion_matrix(y_test, y_pred)

array([[791, 341],
       [480, 627]], dtype=int64)

In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.70      0.66      1132
           1       0.65      0.57      0.60      1107

    accuracy                           0.63      2239
   macro avg       0.64      0.63      0.63      2239
weighted avg       0.63      0.63      0.63      2239



In [37]:
#Hyperparameter optimization
#we are using grid search for hyperparameter optimizing the hyperparameters of the models
param_grid_nb = {'var_smoothing': np.logspace(0, -9, num = 100)}

In [38]:
nb_model_grid = GridSearchCV(estimator = gnb_model, param_grid = param_grid_nb, verbose = 1, cv = 3, n_jobs = -1)

In [39]:
nb_model_grid.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


GridSearchCV(cv=3, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.8480358...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             verbose=1)

In [40]:
nb_model_grid.best_params_

{'var_smoothing': 0.005336699231206307}

In [41]:
y_pred_hyper = nb_model_grid.predict(X_test)

In [42]:
print(confusion_matrix(y_test, y_pred_hyper))

[[870 262]
 [536 571]]


In [43]:
accuracy = accuracy_score(y_test, y_pred_hyper)
print("accuracy after tuning: %.2f%%" % (accuracy*100.0))

accuracy after tuning: 64.36%


In [44]:
print(classification_report(y_test, y_pred_hyper))

              precision    recall  f1-score   support

           0       0.62      0.77      0.69      1132
           1       0.69      0.52      0.59      1107

    accuracy                           0.64      2239
   macro avg       0.65      0.64      0.64      2239
weighted avg       0.65      0.64      0.64      2239



In [45]:
#Lightgbm Classifier
lgb_model = LGBMClassifier(max_depth=3)
lgb_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 4362, number of negative: 4593
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016228 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3086
[LightGBM] [Info] Number of data points in the train set: 8955, number of used features: 1160
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487102 -> initscore=-0.051603
[LightGBM] [Info] Start training from score -0.051603
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

LGBMClassifier(max_depth=3)

In [46]:
y_pred1 = lgb_model.predict(X_test)

In [47]:
accuracy = accuracy_score(y_test, y_pred1)
print("accuracy: %.2f%%" % (accuracy*100.0))

accuracy: 61.28%


In [48]:
confusion_matrix(y_test, y_pred1)

array([[983, 149],
       [718, 389]], dtype=int64)

In [49]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.70      0.66      1132
           1       0.65      0.57      0.60      1107

    accuracy                           0.63      2239
   macro avg       0.64      0.63      0.63      2239
weighted avg       0.63      0.63      0.63      2239



In [50]:
#Hyperparameter optimization
param_grid_lgb = {
              "max_depth": [2, 3, 5, 10],
              "min_child_weight": [0.001, 0.002],
              "learning_rate": [0.05, 0.1]
              }

In [51]:
lgb_grid = GridSearchCV(estimator = lgb_model, param_grid = param_grid_lgb, verbose = 1, cv = 3, n_jobs = -1)

In [52]:
lgb_grid.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 4362, number of negative: 4593
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014508 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3086
[LightGBM] [Info] Number of data points in the train set: 8955, number of used features: 1160
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487102 -> initscore=-0.051603
[LightGBM] [Info] Start training from score -0.051603
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

GridSearchCV(cv=3, estimator=LGBMClassifier(max_depth=3), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1],
                         'max_depth': [2, 3, 5, 10],
                         'min_child_weight': [0.001, 0.002]},
             verbose=1)

In [53]:
GridSearchCV(cv=3, estimator=LGBMClassifier(max_depth=3), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1],
                         'max_depth': [2, 3, 5, 10],
                         'min_child_weight': [0.001, 0.002]},
             verbose=1)

GridSearchCV(cv=3, estimator=LGBMClassifier(max_depth=3), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1],
                         'max_depth': [2, 3, 5, 10],
                         'min_child_weight': [0.001, 0.002]},
             verbose=1)

In [54]:
lgb_grid.best_params_

{'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 0.001}

In [55]:
y_pred1_hyper = lgb_grid.predict(X_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [56]:
accuracy = accuracy_score(y_test, y_pred1_hyper)
print("accuracy after tuning: %.2f%%" % (accuracy*100.0))

accuracy after tuning: 62.89%


In [57]:
print(confusion_matrix(y_test, y_pred1_hyper))

[[906 226]
 [605 502]]


In [58]:
print(classification_report(y_test, y_pred1_hyper))

              precision    recall  f1-score   support

           0       0.60      0.80      0.69      1132
           1       0.69      0.45      0.55      1107

    accuracy                           0.63      2239
   macro avg       0.64      0.63      0.62      2239
weighted avg       0.64      0.63      0.62      2239



In [59]:
#XGBoost Classifier
xgb_model = XGBClassifier(max_depth = 5, min_child_weight = 1)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [60]:
#Evaluation
y_pred2 = xgb_model.predict(X_test)

In [61]:
accuracy = accuracy_score(y_test, y_pred2)
print('Accuracy: %.2f%%' % (accuracy * 100.0))

Accuracy: 63.29%


In [62]:
confusion_matrix(y_test, y_pred2)

array([[874, 258],
       [564, 543]], dtype=int64)

In [63]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.61      0.77      0.68      1132
           1       0.68      0.49      0.57      1107

    accuracy                           0.63      2239
   macro avg       0.64      0.63      0.62      2239
weighted avg       0.64      0.63      0.63      2239



In [64]:
#Hyperparameter optimization
xgb_param_grid = {
              "max_depth": [3, 5],
              "min_child_weight": [1, 2],
              }

In [65]:
xgb_grid = GridSearchCV(estimator = xgb_model, param_grid = xgb_param_grid, verbose = 1, cv = 3, n_jobs = -1)

In [66]:
xgb_grid.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=5,
                                     max_leaves=None, min_child_weight=1,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'max_depth': [3, 5], 'min_child_weight': [1, 2]},
             verbose=1)

In [67]:
xgb_grid.best_params_

{'max_depth': 5, 'min_child_weight': 1}

In [68]:
y_pred2_hyper = xgb_grid.predict(X_test)

In [69]:
accuracy = accuracy_score(y_test, y_pred2_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 63.29%


In [70]:
confusion_matrix(y_test, y_pred2_hyper)

array([[874, 258],
       [564, 543]], dtype=int64)

In [71]:
print(classification_report(y_test, y_pred2_hyper))

              precision    recall  f1-score   support

           0       0.61      0.77      0.68      1132
           1       0.68      0.49      0.57      1107

    accuracy                           0.63      2239
   macro avg       0.64      0.63      0.62      2239
weighted avg       0.64      0.63      0.63      2239

